<a href="https://colab.research.google.com/github/isha060701/RL_WumpusWorld/blob/main/ML_project_WumpusWorld.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -NotebookApp.iopub_data_rate_limit=1.0e10 
import numpy as np

# =============================================================================
# definition of environment and inputs
# =============================================================================
#the shape of the environment
environment_rows = 5 
environment_columns = 5

start = (0, 0) #start point

#hole1 = (2, 0) #hole1 point
#hole2 = (2, 1) #hole2 point
#wall = (3, 2) #wall point

goal = (4,4) #goal point

episodes = 1200 #one sequence of states, actions, and rewards, which ends with a terminal state

epsilon = 0.8 #the percentage of time when we should take the best action (instead of a random action)
discount_factor = 0.5 #discount factor for future rewards

# =============================================================================
# Q-Learning class
# =============================================================================


#q_values = np.zeros((environment_rows, environment_columns, 4))
dq_values = {}
class Q_Learning:
    
    def __init__(self, environment_rows, environment_columns, start, hole1, hole2,hole3, hole4,  goal, episodes, epsilon):
        self.environment_rows = environment_rows
        self.environment_columns = environment_columns
        self.start = start
        self.goal=goal
        self.hole1 = hole1
        self.hole2 = hole2
        self.hole3 = hole3
        self.hole4 = hole4
        # self.hole5 = hole5
        # self.hole6 = hole6
        self.goal = goal
        self.episodes = episodes
        self.epsilon = epsilon
        self.discount_factor = discount_factor
        self.flag=0
        self.flag_test_train=0
        
    def set_matrices(self):
        #create a 2D numpy array to hold the rewards for each state
        self.rewards = np.full((self.environment_rows, self.environment_columns), -1)
        self.visited = np.full((self.environment_rows, self.environment_columns), 0)
        self.rewards[self.hole1] = self.rewards[self.hole2]= self.rewards[self.hole3]= self.rewards[self.hole4]   = -100
        self.rewards[self.goal] = 100
        self.visited[self.hole1] = self.visited[self.hole2]= self.visited[self.hole3]= self.visited[self.hole4]   = 1
        #create a 3D numpy array to hold the current Q-values for each state and action pair: Q(s, a)
        # for i in range(environment_rows):
        #   for j in range(environment_columns):
        #     coordinate=(i,j)
        #     dq_values.update({coordinate:{}})

        #define actions
        #numeric action codes: 0 = up, 1 = right, 2 = down, 3 = left
        self.actions = ['up', 'right', 'down', 'left']
    
    #define a function that determines if the specified location is a terminal state
    def is_terminal_state(self, current_row_index, current_column_index):
        if self.rewards[current_row_index, current_column_index] == -1:
            return False
        else:
            return True

    def get_quad(self,current_row_index,current_column_index):
      quad=0
      if current_row_index<=(int(environment_rows/2)-1) and current_column_index<=(int(environment_columns/2)-1):
        quad=1
      elif current_row_index>(int(environment_rows/2)-1) and current_column_index<=(int(environment_columns/2)-1):
        quad=3
      elif current_column_index>(int(environment_columns/2)-1) and current_row_index<=(int(environment_rows/2)-1):
        quad=2
      elif current_column_index>(int(environment_columns/2)-1) and current_row_index>(int(environment_rows/2)-1):
        quad=4
      return quad

    def append_state(self,current_row_index, current_column_index):
      if current_row_index-1<0:
        rewup=-99999
      else:
        rewup= self.rewards[current_row_index-1,current_column_index]
      if current_column_index+1>=self.environment_columns:
        rewright=-99999
      else:
        rewright=self.rewards[current_row_index,current_column_index+1]
      if current_row_index+1>=self.environment_rows:
        rewdown=-99999
      else:
        rewdown=self.rewards[current_row_index+1,current_column_index]
      if current_column_index-1<0:
        rewleft=-99999
      else:
        rewleft= self.rewards[current_row_index,current_column_index-1]
      
      quad=self.get_quad(current_row_index,current_column_index)
      
      state=(quad,rewup,rewright,rewdown,rewleft)
      #coord=(current_row_index,current_column_index)
      if state not in dq_values:     #if state not present add in table
            if(self.flag_test_train==0):
              dq_values.update({state:[0,0,0,0]})
            elif(self.flag_test_train==1):
              for temp_quad in range(1,5):
                if(temp_quad!=state[0]):
                  temp_state=(temp_quad,state[1],state[2],state[3],state[4])
                  if temp_state in dq_values:
                    return temp_state
              print("not found append_state")
      return state

    #define an epsilon greedy algorithm that will choose which action to take next
    def get_next_action(self, current_row_index, current_column_index, epsilon, episode):
        #if a randomly chosen value between 0 and 1 is less than epsilon, 
        #then choose the most promising value from the Q-table for this state.
        
        state=self.append_state(current_row_index,current_column_index)
        #print(episode)
        #print(dq_values[state])

        # if (episode<400):
          
        if self.flag_test_train==0:  #training
          if np.random.random() < epsilon:   #exploit
              return np.argmax(dq_values[state])
              
          else:                              #explore
              return np.random.randint(4)
        elif self.flag_test_train==1:   #testing
          if state in dq_values:
            return np.argmax(dq_values[state])
          else :
            for temp_quad in range(1,5):
              if(temp_quad!=state[0]):
                temp_state=(temp_quad,state[1],state[2],state[3],state[4])
                if temp_state in dq_values:
                  return np.argmax(dq_values[temp_state])
            print("not found get next action")
              







           

    #define a function that will get the next location based on the chosen action
    def get_next_location(self, current_row_index, current_column_index, action_index):
        new_row_index = current_row_index
        new_column_index = current_column_index
        if self.actions[action_index] == 'up' and current_row_index > 0:
          #print("UP")
          new_row_index -= 1
        elif self.actions[action_index] == 'right' and current_column_index < self.environment_columns - 1:
          #print("RIGHT")
          new_column_index += 1
        elif self.actions[action_index] == 'down' and current_row_index < self.environment_rows - 1:
          #print("DOWN")
          new_row_index += 1
        elif self.actions[action_index] == 'left' and current_column_index > 0:
          #print("LEFT")
          new_column_index -= 1
        # else :
        #   print("terminal")
        return new_row_index, new_column_index


    #run through 500 training episodes
    def train(self):
        for episode in range(self.episodes):
            #get the starting location for this episode
            curr_row_index, curr_column_index = self.start

            #continue taking actions (i.e., moving) until we reach a terminal state
            #(i.e., until we reach the item packaging area or crash into an item storage location)
            while not self.is_terminal_state(curr_row_index, curr_column_index):
      
                #choose which action to take (i.e., where to move next)
                action_index = self.get_next_action(curr_row_index, curr_column_index, self.epsilon,episode)

                #store the old row and column indexes
                old_row_index, old_column_index = curr_row_index, curr_column_index 
    
                #perform the chosen action, and transition to the next state (i.e., move to the next location)
                curr_row_index, curr_column_index = self.get_next_location(curr_row_index, curr_column_index, action_index)
    
                #receive the reward for moving to the new state, and calculate the temporal difference
                

                state_prev=self.append_state(old_row_index, old_column_index)
                state=self.append_state(curr_row_index, curr_column_index)
                reward = state_prev[action_index+1]
                
                if reward==100 and self.flag==0:
                  print("goal found at",(curr_row_index, curr_column_index))
                  if curr_row_index>0:
                    if self.rewards[curr_row_index-1, curr_column_index]==-1:
                      self.rewards[curr_row_index-1, curr_column_index]=90
                  if curr_column_index>0:
                    if self.rewards[curr_row_index, curr_column_index-1]==-1:
                      self.rewards[curr_row_index, curr_column_index-1]=90
                  if curr_row_index< self.environment_rows - 1:
                    if self.rewards[curr_row_index+1, curr_column_index]==-1:
                      self.rewards[curr_row_index+1, curr_column_index]=90
                  if curr_column_index< self.environment_columns - 1:
                    if self.rewards[curr_row_index, curr_column_index+1]==-1:
                      self.rewards[curr_row_index, curr_column_index+1]=90
                  self.flag=1
                  # print("changed")
                  # print(self.rewards)
                
                
                new_q_value= reward + (self.discount_factor * np.max(dq_values[state]))  #Q-learning

                #new_q_value= reward + (self.discount_factor * dq_values[state][action_index])    #SARSA
                
                # print(new_q_value)
                #coord1=(old_row_index,old_column_index)
                #print(dq_values[coord1][state_prev][action_index])
                dq_values[state_prev][action_index]=new_q_value
                # print(dq_values)
        # for i in dq_values:
        #   print (i, dq_values[i])
        # print(len(dq_values))
        print("----------------------")   

#testing
    def get_shortest_path(self, start):
        self.flag_test_train=1
        start_row_index, start_column_index = start
        #return immediately if this is an invalid starting location
        if self.is_terminal_state(start_row_index, start_column_index):
            return []
        else: #if this is a 'legal' starting location
            current_row_index, current_column_index = start_row_index, start_column_index
            shortest_path = []
            shortest_path.append([current_row_index, current_column_index])
            print(current_row_index,current_column_index)

            self.visited[current_row_index, current_column_index]=1;
            while not self.is_terminal_state(current_row_index, current_column_index):
                action_index = self.get_next_action(current_row_index, current_column_index, 1,300)
                
                temp_row,temp_col=self.get_next_location(current_row_index, current_column_index, action_index)

                state=self.append_state(current_row_index,current_column_index)
                temp_q_val= dq_values[state]

                while(self.visited[temp_row,temp_col]==1):   #visited
                  
                  temp_q_val[action_index]=-99999
                  next_highest_index=np.argmax(temp_q_val)
                  if(temp_q_val[next_highest_index]==-99999): #all block
                    return shortest_path
                  temp_row,temp_col=self.get_next_location(current_row_index, current_column_index, next_highest_index)
                current_row_index, current_column_index = temp_row,temp_col
                
                print(current_row_index,current_column_index)

                shortest_path.append([current_row_index, current_column_index])
                self.visited[current_row_index, current_column_index]=1;
        return shortest_path


    def get_board_path(self,start,shortest_path):
      board=self.rewards
      for moves in shortest_path:
        if board[moves[0]][moves[1]]!=100:
          board[moves[0]][moves[1]]=2
        
      for i in range(environment_rows):
        for j in range(environment_columns):
          if board[i][j]==-1:
            print("_",end=' ')
          if board[i][j]==-100:
            print("X",end=' ')
          if board[i][j]==100:
            print("G",end=' ')
          if board[i][j]==2:
            print("O",end=' ')
          if board[i][j]==90:
            print("_",end=' ')
        print("\n")
    

    def get_board(self,start):
      board=self.rewards
      for i in range(environment_rows):
        for j in range(environment_columns):
          if board[i][j]==-1:
            print("_",end=' ')
          if board[i][j]==-100:
            print("X",end=' ')
          if board[i][j]==100:
            print("G",end=' ')
          if board[i][j]==90:
            print("_",end=' ')
        print("\n")



In [ ]:
 # #random but similar boards
# Q_L = Q_Learning(environment_rows, environment_columns, start, (1,3), (1,1), (1,2),(2,1),(2,2), episodes, epsilon)
# Q_L.set_matrices()
# Q_L.rewards
# Q_L.train()
# print("finished training for 1st case")
# Q_L.get_board(start)
# #shortest_path=Q_L.get_shortest_path(start)
# #print(shortest_path)
# print("################################################")
# Q_L1 = Q_Learning(environment_rows, environment_columns, start, (1,2), (1,3), (2,1),(3,1),(2,2), episodes, epsilon)
# Q_L1.set_matrices()
# Q_L1.rewards
# Q_L1.train()
# print("finished training for 2nd case")
# Q_L1.get_board(start)
# #shortest_path1=Q_L1.get_shortest_path(start)
# #print(shortest_path1)
# print("################################################")
# Q_L2 = Q_Learning(environment_rows, environment_columns, start, (0,2), (3,1), (1,2),(2,1),(2,2), episodes, epsilon)
# Q_L2.set_matrices()
# Q_L2.rewards
# Q_L2.train()
# print("finished training for 3rd case")
# Q_L2.get_board(start)
# #shortest_path2=Q_L2.get_shortest_path(start)
# #print(shortest_path1)
# print("################################################")
# Q_L3 = Q_Learning(environment_rows, environment_columns, start, (1,2), (3,0), (0,2),(3,1),(2,2), episodes, epsilon)
# Q_L3.set_matrices()
# Q_L3.rewards
# Q_L3.train()
# print("finished training for 4th case")
# Q_L3.get_board(start)
# #shortest_path3=Q_L3.get_shortest_path(start)
# #print(shortest_path1)
# print("################################################")
# Q_L4 = Q_Learning(environment_rows, environment_columns, start, (1,1), (1,2), (2,1),(4,0),(2,2), episodes, epsilon)
# Q_L4.set_matrices()
# Q_L4.rewards
# Q_L4.train()
# print("finished training for 5th case")
# Q_L4.get_board(start)
# #shortest_path4=Q_L4.get_shortest_path(start)
# #print(shortest_path1)
# print("################################################")
# Q_L6 = Q_Learning(environment_rows, environment_columns, start, (1,1), (1,2), (2,1),(3,1),(2,2), episodes, epsilon)
# Q_L6.set_matrices()
# Q_L6.rewards
# Q_L6.train()
# print("finished training for 6th case")
# Q_L6.get_board(start)
# #shortest_path6=Q_L6.get_shortest_path(start)
# #print(shortest_path1)
# print("################################################")
# Q_L5 = Q_Learning(environment_rows, environment_columns, start, (1,1), (1,2), (0,4),(2,1),(2,2), episodes, epsilon)
# Q_L5.set_matrices()
# Q_L5.rewards
# #Q_L2.train()
# print("Testing for 7th case")
# shortest_path5=Q_L5.get_shortest_path(start)
# print(shortest_path5)
# Q_L5.get_board_path(start,shortest_path5)



Q_L = Q_Learning(environment_rows, environment_columns, start, (0,2), (0,3), (1,4),(2,0),(4,4), episodes, epsilon)
Q_L.set_matrices()
Q_L.rewards
Q_L.train()
print("finished training for 1st case")
Q_L.get_board(start)
#shortest_path=Q_L.get_shortest_path(start)
#print(shortest_path)
print("################################################")

Q_L1 = Q_Learning(environment_rows, environment_columns, start, (0,4), (2,0), (2,1),(3,2),(4,4), episodes, epsilon)
Q_L1.set_matrices()
Q_L1.rewards
Q_L1.train()
print("finished training for 2nd case")
Q_L1.get_board(start)
#shortest_path1=Q_L1.get_shortest_path(start)
#print(shortest_path1)
print("################################################")


Q_L2 = Q_Learning(environment_rows, environment_columns, start, (1,2), (1,4), (2,2),(4,0),(4,4), episodes, epsilon)
Q_L2.set_matrices()
Q_L2.rewards
Q_L2.train()
print("finished training for 3rd case")
Q_L2.get_board(start)
#shortest_path2=Q_L2.get_shortest_path(start)
#print(shortest_path1)
print("################################################")
Q_L3 = Q_Learning(environment_rows, environment_columns, start, (0,2), (2,1), (2,2),(3,1),(4,4), episodes, epsilon)
Q_L3.set_matrices()
Q_L3.rewards
Q_L3.train()
print("finished training for 4th case")
Q_L3.get_board(start)
#shortest_path3=Q_L3.get_shortest_path(start)
#print(shortest_path1)
print("################################################")
Q_L4 = Q_Learning(environment_rows, environment_columns, start, (1,0), (1,1), (1,3),(3,2),(4,4), episodes, epsilon)
Q_L4.set_matrices()
Q_L4.rewards
Q_L4.train()
print("finished training for 5th case")
Q_L4.get_board(start)
#shortest_path4=Q_L4.get_shortest_path(start)
#print(shortest_path1)
print("################################################")
Q_L6 = Q_Learning(environment_rows, environment_columns, start, (0,2), (2,2), (4,0),(3,2),(4,4), episodes, epsilon)
Q_L6.set_matrices()
Q_L6.rewards
Q_L6.train()
print("finished training for 6th case")
Q_L6.get_board(start)
#shortest_path6=Q_L6.get_shortest_path(start)
#print(shortest_path1)
print("################################################")

Q_L5 = Q_Learning(environment_rows, environment_columns, start, (0,2), (2,1), (3,2),(4,3),(4,4),
 episodes, epsilon)
Q_L5.set_matrices()
Q_L5.rewards
#Q_L2.train()
print("Testing for 7th case")
Q_L5.get_board(start)
shortest_path5=Q_L5.get_shortest_path(start)

print(shortest_path5)
Q_L5.get_board_path(start,shortest_path5)




goal found at (4, 4)
----------------------
finished training for 1st case
_ _ X X _ 

_ _ _ _ X 

X _ _ _ _ 

_ _ _ _ _ 

_ _ _ _ G 

################################################
goal found at (4, 4)
----------------------
finished training for 2nd case
_ _ _ _ X 

_ _ _ _ _ 

X X _ _ _ 

_ _ X _ _ 

_ _ _ _ G 

################################################
goal found at (4, 4)
----------------------
finished training for 3rd case
_ _ _ _ _ 

_ _ X _ X 

_ _ X _ _ 

_ _ _ _ _ 

X _ _ _ G 

################################################
goal found at (4, 4)
----------------------
finished training for 4th case
_ _ X _ _ 

_ _ _ _ _ 

_ X X _ _ 

_ X _ _ _ 

_ _ _ _ G 

################################################
goal found at (4, 4)
----------------------
finished training for 5th case
_ _ _ _ _ 

X X _ X _ 

_ _ _ _ _ 

_ _ X _ _ 

_ _ _ _ G 

################################################
goal found at (4, 4)
----------------------
finished training for 6th case
_ _ X